# Match-Up

Given a SENTINEL-3 OLCI, i.e. [S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004](https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004) with specified attributes:

- `geometry`,
- [`properties.start_datetime`, `properties.end_datetime`]

search all active AERONET acquisitions, where:

- `properties.start_datetime` - 1h <= `Date(dd:mm:yyyy),Time(hh:mm:ss)` <= `properties.end_datetime` + 1h
- `Site_Latitude(Degrees),Site_Longitude(Degrees)` in `geometry`

## Execute the `search` operation

In [1]:
from datetime import (
    datetime,
    timedelta,
    timezone
)
from dateutil import parser
from httpx import (
    Client,
    Response
)
from pathlib import Path
from pystac import Item
from typing import Any

import json
import sys

out_dir: Path = Path('.')

with Client() as client:
    response: Response = client.get('https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004')
    item_str: str = response.text
    item_dict = json.loads(item_str)

json.dump(item_dict, sys.stdout, indent=2)

{
  "id": "S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004",
  "bbox": [
    -1.51876,
    39.545,
    18.484,
    52.4535
  ],
  "type": "Feature",
  "links": [
    {
      "rel": "collection",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "parent",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc"
    },
    {
      "rel": "root",
      "type": "application/json",
      "href": "https://stac.dataspace.copernicus.eu/v1/"
    },
    {
      "rel": "self",
      "type": "application/geo+json",
      "href": "https://stac.dataspace.copernicus.eu/v1/collections/sentinel-3-olci-1-efr-ntc/items/S3A_OL_1_EFR____20251031T094707_20251031T095007_20251101T104938_0180_132_136_2160_PS1_O_NT_004"
    },
    {
      "rel": "version-history",
      "href": "https://trace.

In [2]:

source_item: Item = Item.from_dict(item_dict)

def _parse_date_from_properties(date_string_id: str) -> datetime:
    return parser.parse(source_item.properties[date_string_id])

start_datetime = _parse_date_from_properties('start_datetime') - timedelta(hours=1)
end_datetime = _parse_date_from_properties('end_datetime') + timedelta(hours=1)

def _serialize_date(input_date: datetime) -> str:
    dt_utc = input_date.astimezone(timezone.utc)
    return dt_utc.isoformat().replace("+00:00", "Z")

cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(start_datetime)},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": _serialize_date(end_datetime)},
            ],
        },
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                source_item.geometry,
            ],
        }
    ],
}

json.dump(cql2_filter, sys.stdout, indent=2)

{
  "op": "and",
  "args": [
    {
      "op": "eq",
      "args": [
        {
          "property": "data_type"
        },
        "AOD10"
      ]
    },
    {
      "op": "eq",
      "args": [
        {
          "property": "format"
        },
        "csv"
      ]
    },
    {
      "op": "eq",
      "args": [
        {
          "property": "data_format"
        },
        "daily-average"
      ]
    },
    {
      "op": "t_after",
      "args": [
        {
          "property": "time"
        },
        {
          "timestamp": "2025-10-31T08:47:06.893754Z"
        }
      ]
    },
    {
      "op": "t_before",
      "args": [
        {
          "property": "time"
        },
        {
          "timestamp": "2025-10-31T10:50:06.893754Z"
        }
      ]
    },
    {
      "op": "s_intersects",
      "args": [
        {
          "property": "geometry"
        },
        {
          "type": "Polygon",
          "coordinates": [
            [
              [
                -1.51

In [3]:
from pygeofilter_aeronet import aeronet_search

resulting_item: Item = aeronet_search(
    cql2_filter=cql2_filter,
    output_dir=out_dir
)

json.dump(resulting_item.to_dict(), sys.stdout, indent=2)

2025-11-14 16:40:05.028 | SUCCESS  | pygeofilter_aeronet:aeronet_search:230 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-14 16:40:05.046 | SUCCESS  | pygeofilter_aeronet:aeronet_search:238 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/17aea210-3468-488d-99ab-7535d24f5d6f.csv
2025-11-14 16:40:05.063 | SUCCESS  | pygeofilter_aeronet:aeronet_search:254 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/use_cases/17aea210-3468-488d-99ab-7535d24f5d6f.parquet


{
  "type": "Feature",
  "stac_version": "1.1.0",
  "stac_extensions": [],
  "id": "urn:uuid:17aea210-3468-488d-99ab-7535d24f5d6f",
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -170.5644,
          -64.239996
        ],
        [
          174.768143,
          -64.239996
        ],
        [
          174.768143,
          68.661089
        ],
        [
          -170.5644,
          68.661089
        ],
        [
          -170.5644,
          -64.239996
        ]
      ]
    ]
  },
  "bbox": [
    -170.5644,
    -64.239996,
    174.768143,
    68.661089
  ],
  "properties": {
    "datetime": "2025-11-14T16:40:05.098600Z"
  },
  "links": [
    {
      "rel": "related",
      "href": "https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?AOD10=1&if_no_html=1&AVG=20&year=2025&month=10&day=31&hour=8&year2=2025&month2=10&day2=31&hour2=10&lon1=-1.5187599999999999&lat1=39.545&lon2=18.484&lat2=52.45349999999999",
      "type": "text/csv",
      "

## Visualize the results as Data Frame

In [4]:
from geopandas import read_parquet
from geopandas.geodataframe import GeoDataFrame

geoparquet_file: str = resulting_item.get_assets()['geoparquet'].href
geoparquet_data: GeoDataFrame = read_parquet(geoparquet_file)

geoparquet_data

,AERONET_Site,Date(dd:mm:yyyy),Time(hh:mm:ss),Day_of_Year,AOD_1640nm,AOD_1020nm,AOD_870nm,AOD_865nm,AOD_779nm,AOD_675nm,...,N[340-440_Angstrom_Exponent],N[440-675_Angstrom_Exponent[Polar]],Data_Quality_Level,AERONET_Instrument_Number,AERONET_Site_Name,Site_Latitude(Degrees),Site_Longitude(Degrees),Site_Elevation(m),geometry,datetime
0,Tucson,30:10:2025,12:00:00,303,0.022759,0.030613,0.032073,-999.0,-999.0,0.034822,...,99,0,lev10,1171,Tucson,32.233002,-110.953003,779.000,POINT (-110.953 32.233),2025-10-30 12:00:00
1,Tucson,31:10:2025,12:00:00,304,0.060602,0.065395,0.065430,-999.0,-999.0,0.067265,...,96,0,lev10,1171,Tucson,32.233002,-110.953003,779.000,POINT (-110.953 32.233),2025-10-31 12:00:00
2,GSFC,30:10:2025,12:00:00,303,0.007240,0.016739,0.022338,-999.0,-999.0,0.033286,...,32,0,lev10,1072,GSFC,38.992500,-76.839833,87.000,POINT (-76.83983 38.9925),2025-10-30 12:00:00
3,GSFC,31:10:2025,12:00:00,304,0.012323,0.020479,0.024331,-999.0,-999.0,0.031947,...,82,0,lev10,1072,GSFC,38.992500,-76.839833,87.000,POINT (-76.83983 38.9925),2025-10-31 12:00:00
4,Wallops,30:10:2025,12:00:00,303,0.033409,0.050313,0.056961,-999.0,-999.0,0.073485,...,1,0,lev10,1490,Wallops,37.932850,-75.471950,37.000,POINT (-75.47195 37.93285),2025-10-30 12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,Jeonju_RDA,30:10:2025,12:00:00,303,1.320580,1.250719,1.240559,-999.0,-999.0,1.241184,...,37,0,lev10,1535,Jeonju_RDA,35.828371,127.055012,42.174,POINT (127.05501 35.82837),2025-10-30 12:00:00
630,Invercargill_TROPOS,30:10:2025,12:00:00,303,0.043877,0.052294,0.056676,-999.0,-999.0,0.061928,...,18,0,lev10,1378,Invercargill_TROPOS,-46.417479,168.330753,5.000,POINT (168.33075 -46.41748),2025-10-30 12:00:00
631,Invercargill_TROPOS,31:10:2025,12:00:00,304,0.091755,0.093959,0.104787,-999.0,-999.0,0.102952,...,1,0,lev10,1378,Invercargill_TROPOS,-46.417479,168.330753,5.000,POINT (168.33075 -46.41748),2025-10-31 12:00:00
632,CIBA,30:10:2025,12:00:00,303,0.484599,0.444755,0.428063,-999.0,-999.0,0.410291,...,7,0,lev10,383,CIBA,41.814000,-4.932400,839.000,POINT (-4.9324 41.814),2025-10-30 12:00:00


## Visualize results on Map screen

In [6]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from geopandas import GeoSeries
from folium.plugins import (
    Fullscreen
)
from IPython.display import (
    display,
    HTML
)
from pandas import concat
from shapely.geometry import shape
from shapely.geometry.base import BaseGeometry
from stac_geoparquet.arrow import parse_stac_items_to_arrow

geoparquet_data = geoparquet_data.dissolve(by='AERONET_Site')

if source_item.geometry:
    filter_geom: BaseGeometry = shape(source_item.geometry)

    if geoparquet_data.crs:
        filter_series = GeoSeries(
            [filter_geom],
            crs="EPSG:4326"
        ).to_crs(geoparquet_data.crs).iloc[0]

        geoparquet_data = geoparquet_data[geoparquet_data.geometry.within(filter_geom)]

map: Map = Map()
layer_control = LayerControl(position="topright", collapsed=True)
fullscreen = Fullscreen()

GeoJson(
    geoparquet_data.to_json(default=str),
    name="Stac Item footprints",
    style_function=lambda x: {"fillColor": "#00000000", "color": "#0000ff", "weight": 1},
    control=True,
).add_to(map)

GeoJson(
    source_item,
    name="Polygon",
    style_function=lambda f: {
        "fillColor": "yellow",
        "color": "red",
        "weight": 2,
        "fillOpacity": 0.3,
    }
).add_to(map)

layer_control.add_to(map)
fullscreen.add_to(map)
map.fit_bounds(map.get_bounds()) # type: ignore not to important for the demo
map